## Altmap Experiments
### Compare altmap to map eq using networkx

In [48]:
from matplotlib.ticker import (AutoMinorLocator)

from altmap.altmap_helpers.general import *

# show plots in separate window
%pylab

# init rc params
init_plt_params()

def generate_disconnected_cliques(num_cliques=3, clique_size=5):
    N = num_cliques * clique_size
    G = nx.Graph()
    for i in range(num_cliques):
        clique = nx.complete_graph(clique_size)
        clique = nx.convert_node_labels_to_integers(clique, first_label=(1 + i*clique_size))
        G.update(clique)
    
    nodes_ids = list(range(1, N+1))
    labels = [comm+1 for comm in range(num_cliques) for _ in range(clique_size)]
    communities_true = dict(zip(nodes_ids, labels))
    return G, communities_true

# compute essential cost function values for a barbell network with
# given clique size
def node_mvmt_delta(clique_size, number_of_cliques):
    nc = clique_size
    k = number_of_cliques
    
    p_alpha = 1.0 / (nc*k)
    pi = pj = 1.0 / k
    before_mvmt = -2.0 * altmap_module_cost(pi, pi)
    
    pi = pi - p_alpha
    pj = pj + p_alpha
    p_jtoj = 2.0 / (nc - 1) / (k + 1)
    after_mvmt = -altmap_module_cost(pi, pi) - altmap_module_cost(pj, pj*(1.0 - p_jtoj))
    
    return after_mvmt - before_mvmt

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [80]:
n_list = np.logspace(0,3,30, endpoint=False, dtype=int)
n_list = list(map(lambda x: x+x%2, n_list))
n_list = np.unique(n_list) + 3

k_list = [2, 3, 4, 8, 16]

delta_J_arr = np.zeros((len(k_list), len(n_list)))
for i, k in enumerate(k_list):
    print (f'Iteration {i+1}/{len(k_list)}')
    
    delta_list = np.zeros((len(n_list),))
    for j, n in enumerate(n_list):
        delta_list[j] = node_mvmt_delta(n, k)
        
    delta_J_arr[i, :] = delta_list

Iteration 1/5
Iteration 2/5
Iteration 3/5
Iteration 4/5
Iteration 5/5


In [81]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
for i,_ in enumerate(k_list):
    ax.plot(n_list, -delta_J_arr[i,:], 'x--',  label=f'{k_list[i]} cliques')

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$')
ax.set_ylabel(r'$\mathcal{J}(\mathcal{Y}^\bullet) - \mathcal{J}(\mathcal{Y}^\circ)$')
ax.legend()
ax.semilogx()
plt.tight_layout()

In [84]:
fig_path = '../../../../disjoint_cliques.pdf'
plt.savefig(fig_path, dpi=600, format='pdf')

In [42]:
num_cliques = 3
clique_size = 10
G, communities_true = generate_disconnected_cliques(num_cliques, clique_size)

# run community detection
communities_found, num_communities_found,_,_ = infomap(G, altmap=False)
# communities_found, num_communities_found,_,_ = infomap(G, altmap=True)
# communities_found, num_communities_found,_,_ = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

# cost = altmap_cost(G, communities_found)
# print (f'Achieved cost L = {cost}')


OrderedDict([(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2), (19, 2), (20, 2), (21, 3), (22, 3), (23, 3), (24, 3), (25, 3), (26, 3), (27, 3), (28, 3), (29, 3), (30, 3)])
We found 3 communities.


In [109]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)
plt.tight_layout()

In [47]:
# generate network
num_cliques = 12
clique_size = 3
N = num_cliques * clique_size # num nodes
G, communities_true = generate_disconnected_cliques(num_cliques, clique_size)

cost = altmap_cost(G, communities_true)
print (f'Ground truth cost L = {cost}')
print (f'Analytical = {np.log2(num_cliques)}')

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))
cost = altmap_cost(G, communities_init)
print (f'Initial cost L = {cost}')

# independent sets
labels = list(range(1,clique_size+1)) * num_cliques
communities_ind = dict(zip(nodes_ids, labels))
cost = altmap_cost(G, communities_ind)
print (f'Maximal independent sets cost L = {cost}')
print (f'Analytical = {np.log2(clique_size) - np.log2(clique_size-1)}')


Ground truth cost L = -3.584962500721156
Analytical = 3.584962500721156
Initial cost L = -0.04064198449734591
Maximal independent sets cost L = -0.5849625007211564
Analytical = 0.5849625007211561


In [ ]:
plt.close('all')

plt.figure(figsize=(5,8))
drawNetwork(G, communities_true)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_init)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_ind)
plt.tight_layout()

# plt.close('all')
# fig, axs = plt.subplots(1,3)
# fig.suptitle(f'Sample ring of cliques networks')
# 
# drawNetwork(G, communities_true, ax=axs[0])
# axs[0].set_xlabel('Ground truth')
# drawNetwork(G, communities_init, ax=axs[1])
# axs[1].set_xlabel('Initial partition')
# drawNetwork(G, communities_ind, ax=axs[2])
# axs[2].set_xlabel('Maximum independent sets')